In [1]:
import pandas as pd
import numpy as np
from time import sleep
from tqdm import tqdm
import random

from googleapiclient import discovery
import json

In [5]:
# displaying options
pd.set_option('display.float_format', '{:.20f}'.format)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)
# deprecation warning set off
pd.options.mode.chained_assignment = None 

In [3]:
data = pd.read_csv('full_analysis.csv')

In [4]:
data['toxicity'] = 0

In [7]:
 API_KEY = #<YOUR_API_KEY>
client = discovery.build(
  "commentanalyzer",
  "v1alpha1",
  developerKey=API_KEY,
  discoveryServiceUrl="https://commentanalyzer.googleapis.com/$discovery/rest?version=v1alpha1",
  static_discovery=False,
)

In [16]:
def toxicity_detection(sentences):
    toxic = []
    for sent in sentences:
        analyze_request = {
            'comment': { 'text': f"{sent}" },
            'languages' : ["de"],
            'requestedAttributes': {'TOXICITY': {}},
        }

        response = client.comments().analyze(body=analyze_request).execute()
        j = json.dumps(response, indent=2)
        #print(json.loads(j)['attributeScores']['TOXICITY']['summaryScore']['value'])
        toxic.append(json.loads(j)['attributeScores']['TOXICITY']['summaryScore']['value'])
    return sum(toxic)/len(toxic)

In [34]:
#only conspiratorial messages
conspiracy = con2.query('is_CHEM_algo != 0 or is_REICH_algo != 0 or is_PIZZA_algo != 0 or is_Q_algo != 0')

In [31]:
#for i in data.messages_processed:
    
sentences = []
for text in conspiracy.messages_processed:
    tmp = [sent for sent in text.split('  ') if len(sent.split()) > 5]
    if (len(tmp) > 100):
        tmp = random.sample(tmp, 100)
    if (len(tmp) > 5):
        sentences.append(len(tmp))

In [ ]:
tmp = [sent for sent in text.split('  ') if len(sent.split()) > 5]
    if (len(tmp) > 100):
        tmp = random.sample(tmp, 100)
    if (len(tmp) > 20):

In [13]:
n= 10000
list_df = [con2[i:i+n] for i in range(0,len(con2),n)]

In [67]:
final_toxic_list = []
for df in list_df:
    for i in tqdm(range(len(df)-1)):
        row = df.iloc[i]
        toxic = []
        if ((row['is_CHEM_algo']+row['is_REICH_algo']+row['is_Q_algo']+row['is_PIZZA_algo'] > 0) and (row['toxicity'] == 0)): 

            tmp = [sent for sent in row.messages_processed.split('  ') if len(sent.split()) > 5]
            
            if (len(tmp) > 100):
                tmp = random.sample(tmp, 100)
            if (len(tmp) > 1):
                row['toxicity'] = toxicity_detection(tmp)
                #row['toxicity'] = len(toxic) and sum(toxic)/len(toxic) or 0
                
        df.iloc[i] = row
        
        
        
        #df.at[i, 'toxicity'] = toxic
    final_toxic_list.append(df)

100%|███████████████████████████████████████████████████████████████████████████████| 4544/4544 [01:04<00:00, 70.16it/s]


In [68]:
con = pd.concat(final_toxic_list)

In [233]:
con.to_csv('fa_toxic.csv')